In [1]:
# @title Import libraries
import pandas as pd
import re
import requests

In [2]:
# Scores data contains question metadata
scores = pd.read_csv(f'https://data.heroku.com/dataclips/nudnpycciffydoeihwbtttlkwpcj.csv')

fabQuestions = pd.read_csv('scores/Q4 2024 FAB - questions list - FINAL BOT.csv')

In [3]:
# Show me scores where the question_title contains "influenza"
scores[scores['question_title'].str.contains('influenza', na=False)]

,question_id,question_title,created_at,user_id,score,score_type,forecaster,is_bot,resolution,scheduled_close_time,actual_close_time,question_weight,cp_reveal_time,post_id
22041,29461,Will the weekly total number of influenza hosp...,2024-11-05 14:24:12.630515+00,182315,36.905370,peer,RyansAGI,True,yes,2024-11-06 15:30:00+00,2024-11-06 15:30:00+00,0.5283,2024-11-06 15:30:00+00,29589
22042,29461,Will the weekly total number of influenza hosp...,2024-11-05 14:24:12.630515+00,182315,0.317036,relative_legacy,RyansAGI,True,yes,2024-11-06 15:30:00+00,2024-11-06 15:30:00+00,0.5283,2024-11-06 15:30:00+00,29589
22043,29461,Will the weekly total number of influenza hosp...,2024-11-05 14:24:12.630515+00,182315,47.778783,baseline,RyansAGI,True,yes,2024-11-06 15:30:00+00,2024-11-06 15:30:00+00,0.5283,2024-11-06 15:30:00+00,29589
22044,29461,Will the weekly total number of influenza hosp...,2024-11-05 14:24:12.630515+00,182315,43.902960,spot_peer,RyansAGI,True,yes,2024-11-06 15:30:00+00,2024-11-06 15:30:00+00,0.5283,2024-11-06 15:30:00+00,29589
22045,29461,Will the weekly total number of influenza hosp...,2024-11-05 14:24:12.630515+00,182315,54.596837,spot_baseline,RyansAGI,True,yes,2024-11-06 15:30:00+00,2024-11-06 15:30:00+00,0.5283,2024-11-06 15:30:00+00,29589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36901,29941,Will the USDA's Center for Veterinary Biologic...,2024-11-15 14:08:43.489443+00,222631,-4.922842,spot_peer,Grizeu_Bot,True,no,2024-11-16 15:30:00+00,2024-11-16 15:30:00+00,1.0000,2024-11-16 15:30:00+00,30111
36902,29941,Will the USDA's Center for Veterinary Biologic...,2024-11-15 14:08:43.489443+00,222631,37.851162,spot_baseline,Grizeu_Bot,True,no,2024-11-16 15:30:00+00,2024-11-16 15:30:00+00,1.0000,2024-11-16 15:30:00+00,30111
36903,29941,Will the USDA's Center for Veterinary Biologic...,2024-11-15 14:08:43.489443+00,222631,11.298073,baseline,Grizeu_Bot,True,no,2024-11-16 15:30:00+00,2024-11-16 15:30:00+00,1.0000,2024-11-16 15:30:00+00,30111
36904,29941,Will the USDA's Center for Veterinary Biologic...,2024-11-15 14:08:43.489443+00,222631,-0.039444,relative_legacy,Grizeu_Bot,True,no,2024-11-16 15:30:00+00,2024-11-16 15:30:00+00,1.0000,2024-11-16 15:30:00+00,30111


In [4]:
# Rename first column to 'link'
fabQuestions = fabQuestions.rename(columns={fabQuestions.columns[0]: 'link', 'title': 'question_title'})
scores = scores.rename(columns={'question_id': 'bot_question_id'})

In [5]:
def add_question_ids(df):
    def extract_metaculus_id(url):
        if pd.isna(url):
            return None
        pattern = r'/questions/(\d+)/'
        match = re.search(pattern, str(url))
        return match.group(1) if match else None
    
    # Create new column with extracted IDs
    df['cp_post_id'] = df['link'].apply(extract_metaculus_id)
    return df

fabQuestions = add_question_ids(fabQuestions)

# Take only the questions that were also on main feed (cp_post_id not None)
fabQuestions = fabQuestions[~fabQuestions['cp_post_id'].isna()]

In [6]:
# Merge scores and fabQuestions on question title
merged = scores[['bot_question_id', 'question_title', 'cp_reveal_time', 'resolution']].merge(fabQuestions[['question_title', 'cp_post_id']], on='question_title', how='right')

# Drop duplicates
merged = merged.drop_duplicates()

# Cast merged['bot_question_id'] to int
merged['bot_question_id'] = merged['bot_question_id'].astype('Int64')

In [7]:
# Show head of merged data where cp_question_id is not null
merged[~merged['cp_post_id'].isnull()].head()

,bot_question_id,question_title,cp_reveal_time,resolution,cp_post_id
0,28922,Will a senolytic therapy be approved for comme...,2024-10-22 14:30:00+00,no,1621
140,28923,Will a nuclear weapon be detonated as an act o...,2024-10-22 14:30:00+00,no,4779
285,28924,Will a book written by a language model make t...,2024-10-22 14:30:00+00,no,5587
425,28935,"On October 31, 2024, will Nvidia's market capi...",2024-10-22 14:30:00+00,no,24806
570,28959,Will the Liberal Democratic Party win a majori...,2024-10-23 14:30:00+00,no,28972


In [8]:
# Did every cp_question_id get a bot_question_id? Show me rows where bot_question_id is null
nomatch = merged[merged['bot_question_id'].isnull()]

nomatch.shape

(15, 5)

In [9]:
merged.shape

(154, 5)

In [10]:
print(f'no match shape: {nomatch.shape}')
nomatch # I think these don't match because they just wound up not being used in FAB

no match shape: (15, 5)


,bot_question_id,question_title,cp_reveal_time,resolution,cp_post_id
5055,<NA>,Will the US State Department approve more than...,NaN,NaN,29028
6126,<NA>,Will BirdCast report 1 billion birds flying ov...,NaN,NaN,28704
7942,<NA>,"Before December 1, 2024, will laws be in place...",NaN,NaN,2788
12383,<NA>,Will the US sanction Venezuelan oil again by 2...,NaN,NaN,20557
16869,<NA>,Will Ukraine use ATACMS to strike deep within ...,NaN,NaN,30251
17760,<NA>,Will Elena Lasconi win the second round of the...,NaN,NaN,30472
17761,<NA>,Will Calin Georgescu win the second round of t...,NaN,NaN,30472
18047,<NA>,Will a ceasefire between Israel and Lebanon/He...,NaN,NaN,30476
20503,<NA>,"Will Climeworks still exist on December 31, 20...",NaN,NaN,4862
20794,<NA>,Will the undiagnosed disease currently spreadi...,NaN,NaN,30870


In [11]:
# Save merged data to csv
merged.to_csv('bot_to_main_feed_ids.csv', index=False)

In [12]:
# Question title containing 'Iceland' ?
merged[merged['question_title'].str.contains('Iceland')]

,bot_question_id,question_title,cp_reveal_time,resolution,cp_post_id
11178,29718,Will the new Prime Minister of Iceland belong ...,2024-11-12 15:30:00+00,no,29807
11323,29719,Will the new Prime Minister of Iceland belong ...,2024-11-12 15:30:00+00,no,29807
11468,29720,Will the new Prime Minister of Iceland belong ...,2024-11-12 15:30:00+00,no,29807
11623,29721,Will the new Prime Minister of Iceland belong ...,2024-11-12 15:30:00+00,yes,29807
